In [ ]:
import os
import shutil
import time
import netCDF4 as nc

# test en local: 

In [ ]:
#file_path = "D:/GCxGC_MS/DATA/Centroid/R-L-010-817823-Tedlar.cdf"
#file_path = "D:/GCxGC_MS/DATA/test_adeline/A-F-028-817822-droite-ReCIVA.cdf"
#file_path = "/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf"
#A-F-028-817822-droite-ReCIVA.cdf
#file_path = "D:/GCxGC_MS/DATA/test_adeline/J-A-034-751325-Tedlar.cdf"
file_path = "D:/GCxGC_MS/DATA/test_adeline/P-L-007-801838-Tedlar.cdf"


In [ ]:
ds = nc.Dataset(file_path)

In [ ]:
t0 = time.time()
mv = ds["mass_values"][:]
iv = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")
print(mv)

# test sur docker

In [ ]:
#file_path = "/app/data/test_adeline/A-F-028-817822-droite-ReCIVA.cdf"
#file_path ="/app/data/test_adeline/J-A-034-751325-Tedlar.cdf"
file_path ="/app/data/test_adeline/P-L-007-801838-Tedlar.cdf"
print(file_path)

In [ ]:
ds = nc.Dataset(file_path)

In [ ]:
t0 = time.time()
print("start")
mv = ds["mass_values"][:]
iv = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")

# test copie sur docker avant d analyser

In [ ]:
source_path = file_path = "D:/GCxGC_MS/DATA/624848_0017VE_J0.cdf"
tmp_path = "/app/data/tmp" 
if not os.path.exists(tmp_path):
            os.makedirs(tmp_path)
            print(f"Created output directory: {tmp_path}")
 

In [ ]:
if not os.path.exists(tmp_path):
    print(f"Copying to fast access: {tmp_path}")
    t0 = time.time()
    shutil.copy(source_path, tmp_path)
    print("Time to copy:", time.time() - t0, "s")
else:
    print(f"Using cached copy: {tmp_path}")

In [ ]:
from netCDF4 import Dataset

t0 = time.time()
ds = Dataset(tmp_path)
mass_values = ds["mass_values"][:]
intensity_values = ds["intensity_values"][:]
print("Time to load variables:", time.time() - t0, "s")

# test a partir d un .npy


In [ ]:
import numpy as np
import time

In [ ]:
# file_path = "D:/Dossiers Persos/Adeline/Python-2DGC-Alignment/interface_flask/converted_data/624848_0017VE_J0.npy"
#file_path = "/app/data/P-L-007-801838-Tedlar.npy"
#file_path = "/app/data/A-F-028-817822-droite-ReCIVA.npy"
file_path = "/app/data/J-A-034-751325-Tedlar.npy"
print(file_path)

In [ ]:
t0 = time.time()
print("start")
mass_values = np.load(file_path, allow_pickle=True).item()["mass_values"]
iv = np.load(file_path, allow_pickle=True).item()["intensity_values"]
print("Time to load variables:", time.time() - t0, "s")
print(mass_values)

# test a partir dun .h5

In [ ]:
# !pip install h5py
import h5py
import math
import time

In [ ]:

file_path = "/home/camille/Documents/app/data/J-A-034-751325-Tedlar.h5"
# file_path = "/app/data/P-L-007-801838-Tedlar.h5"
#file_path = "/app/data/A-F-028-817822-droite-ReCIVA.h5"
# file_path = "/app/data/J-A-034-751325-Tedlar.h5"

In [ ]:
t0 = time.time()
print("start")
with h5py.File(file_path, 'r') as f:
    mass_values = f['mass_values'][:]      # Charge tout
    iv = f['intensity_values'][:]
print("Time to load variables:", time.time() - t0, "s")
print(mass_values)


In [ ]:
t0 = time.time()
print("start")
with h5py.File(file_path, 'r') as f:
        tic_chromato = f['total_intensity'][:]
        timepara = f["scan_acquisition_time"][np.abs(f["point_count"])
                                              < np.iinfo(np.int32).max]
        
        mv = f["mass_values"][:]
        iv = f["intensity_values"][:]
        range_min = math.ceil(f["mass_range_min"][:].min())
        range_max = math.floor(f["mass_range_max"][:].max())
        start_time = f['scan_acquisition_time'][0] / 60
        end_time = f['scan_acquisition_time'][-1] / 60
print("Time to load variables:", time.time() - t0, "s")
print(mass_values)


In [ ]:
import math
import h5py
import numpy as np
import netCDF4 as nc

In [ ]:
file_path_h5 = "/home/camille/Documents/app/data/J-A-034-751325-Tedlar.h5"
file_path_cdf = "/home/camille/Documents/app/data/J-A-034-751325-Tedlar.cdf"
mod_time = 1.25



In [ ]:
def read_chroma_h5(filename, mod_time, max_val=None):
    if not filename.endswith(".h5"):
        raise ValueError("The file must be a .h5")
    with h5py.File(filename, 'r') as f:
        tic_chromato = f['total_intensity'][:]
        
        # abs_point_count = np.abs(f["point_count"][:])
        # print("abs_point_count", abs_point_count)
        # abs_point_count = np.abs(f["point_count"])
        # print("abs_point_count", abs_point_count)

        timepara = f["scan_acquisition_time"][np.abs(f["point_count"]) < np.iinfo(np.int32).max]
        # print("scan_acquisition_time", scan_acquisition_time)
        print("timepara", timepara)
        if (max_val):
            mv = f["mass_values"][:max_val]
            iv = f["intensity_values"][:max_val]
            
        else:
            mv = f["mass_values"][:]
            print("mv", mv)
            iv = f["intensity_values"][:]
            print("iv", iv)
        range_min = math.ceil(f["mass_range_min"][:].min())
        print("range_min", range_min)
        range_max = math.floor(f["mass_range_max"][:].max())
        print("range_max", range_max)
        start_time = f['scan_acquisition_time'][0] / 60
        print("start_time", start_time)
        end_time = f['scan_acquisition_time'][-1] / 60
        print("end_time", end_time)

    # timepara = scan_acquisition_time[abs_point_count <
    #                                  np.iinfo(np.int32).max]
    # print("timepara", timepara)
    # taux d'échantillonnage : le nombre d'échantillons (points) par unité de temps (par exemple, en Hz).
    sam_rate = 1 / np.mean(timepara[1:] - timepara[:-1])
    l1 = math.floor(sam_rate * mod_time)
    l2 = math.floor(len(tic_chromato) / l1)
    tic_chromato = np.reshape(tic_chromato[:l1*l2], (l2, l1))
    print("l1", l1)
    print("l2", l2)
    print("tic_chromato shape", tic_chromato.shape)
    print("tic_chromato", tic_chromato)

    return (tic_chromato, (start_time, end_time),
            (l1, l2, mv, iv, range_min, range_max))

In [ ]:
def read_chroma_cdf(filename, mod_time, max_val=None):

    if not filename.endswith(".cdf"):
        raise ValueError("The file must be a .cdf")

    ds = nc.Dataset(filename, encoding="latin-1")
    tic_chromato = ds['total_intensity']
    abs_point_count = np.abs(ds["point_count"])
    print("abs_point_count", abs_point_count)
    # scan_acquisition_time = ds["scan_acquisition_time"]
    Timepara = ds["scan_acquisition_time"][abs_point_count < np.iinfo(np.int32).max]
    # print("scan_acquisition_time", scan_acquisition_time)
    print("Timepara", Timepara)
    
    # taux d'échantillonnage : le nombre d'échantillons (points) par unité de temps (par exemple, en Hz).
    sam_rate = 1 / np.mean(Timepara[1:] - Timepara[:-1])
    l1 = math.floor(sam_rate * mod_time)
    l2 = math.floor(len(tic_chromato) / l1)

    if (max_val):
        mv = ds["mass_values"][:max_val]
        iv = ds["intensity_values"][:max_val]
    else:
        mv = ds["mass_values"][:]
        print("mv", mv)
        iv = ds["intensity_values"][:]
        print("iv", iv)
    range_min = math.ceil(ds["mass_range_min"][:].min())
    print("range_min", range_min)
    range_max = math.floor(ds["mass_range_max"][:].max())
    print("range_max", range_max)

    tic_chromato = np.reshape(tic_chromato[:l1*l2], (l2, l1))
    print("tic_chromato", tic_chromato)

    start_time = ds['scan_acquisition_time'][0] / 60
    print("start_time", start_time)
    end_time = ds['scan_acquisition_time'][-1] / 60
    print("end_time", end_time)
    print("l1", l1)
    print("l2", l2)
    print("tic_chromato shape", tic_chromato.shape)

    return (tic_chromato, (start_time, end_time),
            (l1, l2, mv, iv, range_min, range_max))


In [ ]:
read_chroma_h5(file_path_h5, mod_time, max_val=1000)


In [ ]:
read_chroma_cdf(file_path_cdf, mod_time, max_val=1000)